# Calculating ranking functions
For each model calculate average metrics for all the splits of each the model, or the ranking/ regret of the model when compared with the others in each split

In [1]:
import pandas as pd
import numpy as np
import json 
import os
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [2]:
from IPython.display import display, HTML


In [71]:
# DataFrame.rolling -> simple moving average
# Weighted moving average sum(w*x) / sum(w)
# Exponential moving average
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

#these are calculated for all splits of each model
def ranking_functions(df):
    x = df.precision_test
    average = x.mean()
    #-> alfa = 0.25,0.5,0.75
    try:
        mean_invstd_25  = 0.25*x.mean() - (1-0.25)*(x.std()) 
    except:
        mean_invstd_25  = 0.25*x.mean()
    try:
        mean_invstd_50  = 0.5*x.mean() - (1-0.5)*(x.std() )
    except:
        mean_invstd_50  = 0.5*x.mean()
    try:
        mean_invstd_75  = 0.75*x.mean() - (1-0.75)*(x.std())
    except:
        mean_invstd_75  = 0.75*x.mean() 
    try:
        invstdev = -(x.std())
    except:
        invstdev = 0

    w = [0,1,2,3,4,5,6,7,8,9]
    weighted_avg = sum(w*x) / sum(w)
    ewm = x.ewm(span=10).mean().mean()
    return pd.Series((average,mean_invstd_25,mean_invstd_50,mean_invstd_75,invstdev,weighted_avg),index=['average','average_neg_std_25','average_neg_std_50','average_neg_std_75','neg_stdev','weighted_avg'])


#The regret and rank must be calculated for each split of all models
#Rank: for the highest precision => rank =1. next => rank =2, etc.
#Regret:for the first split, who has the highest precision? (high_prec)
#for that, the regret is 0. for the next one is (prec_2 - high_prec), etc.
def ranking_rank(df):
    df = pd.pivot_table(df, values='precision_test', index=['model','param_config'], columns='time')
    #display(df)
    rank = df.rank()
    #display(rank)
    return  rank.T

def ranking_regret(df):
    df = pd.pivot_table(df, values='precision_test', index=['model','param_config'], columns='time')
    #display(df)
    highest_metric = df.max()
    
    regret_row = (df-highest_metric).T
    #display(regret_row)
    return  regret_row

In [72]:
outputs_metrics.model.value_counts()

XGBClassifier_date_params             500
DecisionTreeClassifier_date_params    500
RandomForestClassifier_date_params    500
MLPClassifier_date_params             500
LogisticRegression_date_params        500
lgb_LGBMClassifier_date_params        500
SVC_date_params                       500
Name: model, dtype: int64

In [73]:
outputs_metrics

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time
0,100,0.555556,0.441926,0.366391,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,0
1,450,0.485549,0.447646,0.472284,"(1, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,1
2,800,0.424198,0.397163,0.497674,"(2, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,2
3,1150,0.516340,0.439252,0.411141,"(3, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,3
4,1500,0.666667,0.424479,0.464646,"(4, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,4
...,...,...,...,...,...,...,...,...
3495,1949,0.635021,0.680672,0.659864,"(5, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,5
3496,2299,0.638066,0.728997,0.679104,"(6, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,6
3497,2649,0.648194,0.597260,0.774390,"(7, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,7
3498,2999,0.657300,0.534274,0.683794,"(8, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,8


In [74]:
outputs_metrics = pd.read_csv("data/outputs_metrics_refined.csv")
outputs_metrics.model.value_counts()
outputs_metrics.sort_values('precision_test',ascending=False).iloc[:50]

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time
2068,3106,0.377946,1.000000,0.000000,"(8, 6, 14)_SVC_date_params.csv",SVC_date_params,14,8
1184,1668,0.445829,1.000000,0.000000,"(4, 5, 25)_MLPClassifier_date_params.csv",MLPClassifier_date_params,25,4
2341,684,0.265362,1.000000,0.000000,"(1, 6, 4)_SVC_date_params.csv",SVC_date_params,4,1
2337,2783,0.367725,1.000000,0.500000,"(7, 6, 39)_SVC_date_params.csv",SVC_date_params,39,7
2324,1732,0.374140,1.000000,1.000000,"(4, 6, 38)_SVC_date_params.csv",SVC_date_params,38,4
2278,3127,0.378003,1.000000,0.000000,"(8, 6, 33)_SVC_date_params.csv",SVC_date_params,33,8
2161,666,0.265350,1.000000,0.000000,"(1, 6, 23)_SVC_date_params.csv",SVC_date_params,23,1
2127,2762,0.367747,1.000000,0.000000,"(7, 6, 2)_SVC_date_params.csv",SVC_date_params,2,7
2118,3111,0.377982,1.000000,0.000000,"(8, 6, 19)_SVC_date_params.csv",SVC_date_params,19,8
2114,1711,0.374130,1.000000,1.000000,"(4, 6, 19)_SVC_date_params.csv",SVC_date_params,19,4


In [81]:
pd.set_option('display.max_rows', 500)

ranking_regret_average = outputs_metrics.groupby(['time']).apply(ranking_regret).T.mean(axis=1)
ranking_regret_average.name = 'regret_average'
#print(ranking_regret_average)

ranking_regret_std = -outputs_metrics.groupby(['time']).apply(ranking_regret).T.std(axis=1)
ranking_regret_std.name = 'regret_neg_std'
#print(ranking_regret_std)

ranking_regret_invstd = 1/outputs_metrics.groupby(['time']).apply(ranking_regret).T.std(axis=1)

ranking_regret_invstd.name = 'regret_invstd'
#print(ranking_regret_std)



mean_invstd_regret_25  = 0.25*ranking_regret_average + (1-0.25)*(ranking_regret_std)
mean_invstd_regret_25.name = 'mean_neg_std_regret_25'
mean_invstd_regret_50  = 0.5*ranking_regret_average + (1-0.5)*(ranking_regret_std)
mean_invstd_regret_50.name = 'mean_neg_std_regret_50'
mean_invstd_regret_75  = 0.75*ranking_regret_average + (1-0.75)*(ranking_regret_std)
mean_invstd_regret_75.name = 'mean_neg_std_regret_75'

ranking_rank_average = outputs_metrics.groupby(['time']).apply(ranking_rank).T.mean(axis=1)
ranking_rank_average.name = 'rank_average'

ranking_rank_std = -outputs_metrics.groupby(['time']).apply(ranking_rank).T.std(axis=1)
ranking_rank_std.name = 'rank_neg_std'
print(ranking_rank_std)
ranking_rank_invstd = 1/outputs_metrics.groupby(['time']).apply(ranking_rank).T.std(axis=1)
ranking_rank_invstd.name = 'rank_invstd'

mean_invstd_rank_25  = 0.25*ranking_rank_average + (1-0.25)*(ranking_rank_std)
mean_invstd_rank_25.name = 'mean_neg_std_rank_25'

mean_invstd_rank_50  = 0.5*ranking_rank_average + (1-0.5)*(ranking_rank_std)
mean_invstd_rank_50.name = 'mean_neg_std_rank_50'

mean_invstd_rank_75  = 0.75*ranking_rank_average + (1-0.75)*(ranking_rank_std)
mean_invstd_rank_75.name = 'mean_neg_std_rank_75'


metrics_p_modelconifg = outputs_metrics.groupby(['model','param_config']).apply(ranking_functions)


model                               param_config
DecisionTreeClassifier_date_params  0               -23.861871
                                    1                -2.371123
                                    2               -25.691546
                                    3                -2.371123
                                    4               -33.778733
                                    5               -35.267391
                                    6              -104.407801
                                    7                -2.371123
                                    8               -33.103206
                                    9               -26.462762
                                    10              -35.769167
                                    11              -47.679952
                                    12              -68.715213
                                    13               -2.371123
                                    14              -62.545561
      

In [82]:
ranking_metrics = pd.concat([metrics_p_modelconifg,\
                             ranking_regret_average,
                             ranking_regret_std,
                             mean_invstd_regret_25,
                            mean_invstd_regret_50,
                            mean_invstd_regret_75,
                             ranking_rank_average,
                            ranking_rank_std,
                            mean_invstd_rank_25,
                            mean_invstd_rank_50,
                            mean_invstd_rank_75], axis=1)\
                            .replace(to_replace=np.inf,value = 0,)
ranking_metrics

average  average_neg_std_25  \
model                              param_config                                 
DecisionTreeClassifier_date_params 0             0.410483            0.070350   
                                   1             0.000000            0.000000   
                                   2             0.440542            0.070704   
                                   3             0.000000            0.000000   
                                   4             0.409048            0.051623   
                                   5             0.286952           -0.079909   
                                   6             0.445182            0.012366   
                                   7             0.000000            0.000000   
                                   8             0.477350            0.075258   
                                   9             0.073080           -0.097302   
                                   10            0.459319            0.040767   
                                   11            0.460737            0.073154   
                                   12            0.454358           -0.016159   
                                   13            0.000000            0.000000   
                                   14            0.285133           -0.112935   
                                   15            0.000000            0.000000   
                                   16            0.468990            0.077968   
                                   17            0.055072           -0.116848   
                                   18            0.418888            0.061367   
                                   19            0.093421           -0.132922   
                                   20            0.430241            0.051435   
                                   21            0.000000            0.000000   
                                   22            0.025664           -0.054451   
                                   23            0.407151            0.060621   
                                   24            0.455747            0.070152   
                                   25            0.457259            0.069895   
                                   26            0.415488            0.065915   
                                   27            0.317051           -0.088925   
                                   28            0.455545            0.065671   
                                   29            0.515220            0.070996   
                                   30            0.000000            0.000000   
                                   31            0.039216           -0.083204   
                                   32            0.401716           -0.015236   
                                   33            0.461238            0.070636   
                                   34            0.459678            0.071120   
                                   35            0.465176            0.069488   
                                   36            0.045639           -0.096832   
                                   37            0.451769            0.080704   
                                   38            0.434614            0.072133   
                                   39            0.407863            0.061307   
                                   40            0.420776            0.056081   
                                   41            0.445724            0.060655   
                                   42            0.025664           -0.054451   
                                   43            0.147159           -0.121812   
                                   44            0.464251            0.074934   
                                   45            0.413369            0.066466   
                                   46            0.000000            0.000000   
                                   47            0.468200            0.064741   


In [83]:
ranking_metrics.to_csv('ranking_metrics.csv')

In [84]:
best_config_p_metric =   ranking_metrics.idxmax()\
#    .groupby('model').apply(idxmax_param_config)

best_config_p_metric

average                           (XGBClassifier_date_params, 32)
average_neg_std_25                (XGBClassifier_date_params, 32)
average_neg_std_50                (XGBClassifier_date_params, 32)
average_neg_std_75                (XGBClassifier_date_params, 32)
neg_stdev                 (DecisionTreeClassifier_date_params, 1)
weighted_avg                      (XGBClassifier_date_params, 35)
regret_average                    (XGBClassifier_date_params, 32)
regret_neg_std               (lgb_LGBMClassifier_date_params, 31)
mean_neg_std_regret_25            (XGBClassifier_date_params, 49)
mean_neg_std_regret_50        (lgb_LGBMClassifier_date_params, 2)
mean_neg_std_regret_75            (XGBClassifier_date_params, 32)
rank_average                 (lgb_LGBMClassifier_date_params, 30)
rank_neg_std              (DecisionTreeClassifier_date_params, 1)
mean_neg_std_rank_25         (lgb_LGBMClassifier_date_params, 30)
mean_neg_std_rank_50         (lgb_LGBMClassifier_date_params, 30)
mean_neg_s

In [85]:
model_series = []
params_series = []
for _, model_config in best_config_p_metric.iteritems():
    print(model_config)
    model = model_config[0]
    param_config =  model_config[1] 
    model_series.append(model)
    params_series.append(param_config)

best_config_p_metric = best_config_p_metric.to_frame().drop(0,axis=1)
best_config_p_metric['model'] = model_series
best_config_p_metric['param_config'] = params_series
best_config_p_metric

('XGBClassifier_date_params', 32)
('XGBClassifier_date_params', 32)
('XGBClassifier_date_params', 32)
('XGBClassifier_date_params', 32)
('DecisionTreeClassifier_date_params', 1)
('XGBClassifier_date_params', 35)
('XGBClassifier_date_params', 32)
('lgb_LGBMClassifier_date_params', 31)
('XGBClassifier_date_params', 49)
('lgb_LGBMClassifier_date_params', 2)
('XGBClassifier_date_params', 32)
('lgb_LGBMClassifier_date_params', 30)
('DecisionTreeClassifier_date_params', 1)
('lgb_LGBMClassifier_date_params', 30)
('lgb_LGBMClassifier_date_params', 30)
('lgb_LGBMClassifier_date_params', 30)


,model,param_config
average,XGBClassifier_date_params,32
average_neg_std_25,XGBClassifier_date_params,32
average_neg_std_50,XGBClassifier_date_params,32
average_neg_std_75,XGBClassifier_date_params,32
neg_stdev,DecisionTreeClassifier_date_params,1
weighted_avg,XGBClassifier_date_params,35
regret_average,XGBClassifier_date_params,32
regret_neg_std,lgb_LGBMClassifier_date_params,31
mean_neg_std_regret_25,XGBClassifier_date_params,49
mean_neg_std_regret_50,lgb_LGBMClassifier_date_params,2


In [86]:
best_config_p_metric.to_csv('data/best_model_p_func.csv')

In [226]:
X_val.to_csv('data/X_validation.csv')
y_val.to_csv('data/y_validation.csv')

In [227]:
best_config_p_metric

average                    (XGBClassifier, 20)
average_std_25             (MLPClassifier, 17)
average_std_50             (MLPClassifier, 17)
average_std_75    (RandomForestClassifier, 25)
stdev                      (MLPClassifier, 16)
weighted_avg               (XGBClassifier, 20)
ewm_              (RandomForestClassifier, 37)
regret_average             (XGBClassifier, 20)
rank_average      (RandomForestClassifier, 34)
dtype: object

In [239]:
from sklearn.metrics import accuracy_score, recall_score

def find_threshold(y_true_th,y_proba_th, metric_1, metric_2, min_metric_2= 0.05,maximize_metric_2 = False):
    
    min_true_for_metric_1 = y_true_th.sum()*min_metric_2
    y_proba_cum = pd.DataFrame([y_proba_th,y_true_th],index=['y_proba_th','y_true_th']).T\
        .sort_values('y_proba_th',ascending = False)
    y_proba_cum['cumulative'] = y_proba_cum.y_true_th.cumsum()
    
    min_threshold = y_proba_cum.query(f'cumulative >= {min_true_for_metric_1}').iloc[0].y_proba_th
    metric = metric_1(y_true_th,y_proba_th>min_threshold)
    return min_threshold,metric
    
clfs = [NeuralNetClassifier,
         RandomForestClassifier,
        LogisticRegression,
       DecisionTreeClassifier,
        lgb.LGBMClassifier,
        XGBClassifier,
        MLPClassifier,
        SVC
       ]
clfs_names = ['NeuralNetClassifier',
         'RandomForestClassifier',
        'LogisticRegression',
       'DecisionTreeClassifier',
        "lgb_LGBMClassifier",
        'XGBClassifier',
        'MLPClassifier',
        'SVC'
       ]



clfs_names_dict = dict(zip(clfs_names,clfs))


def get_metrics_prod(file_test,model_name_test,y_true):
    #get any dict, they are all the same
    params_test = pd.read_csv(file_test).get_p.iloc[0].replace("nan", "None")
    


    #Create best model config acording to avg
    best_model_config_test = clfs_names_dict[model_name_test](**eval(params_test))
    
    #train on the whole train
    best_model_config_test = best_model_config_test.fit(X_train,y_train)
    y_prod_pred = best_model_config_test.predict_proba(X_test)[:,1]
    
    th,metric = find_threshold(y_true,y_prod_pred, precision_score, recall_score, min_metric_2= 0.05)
    
    print(np.array(y_prod_pred))
    print(np.array(y_true))
    return th, accuracy_score(y_true,y_prod_pred>th),best_model_config_test
    
def get_test_threshold(model,param_config):
    #get any filename to get the params
    file_test = 'outputs_models3/'
    data_model_config = outputs_metrics.query(f"model =='{model}' & param_config == {param_config}")
    print(data_model_config)
    file_test =file_test + data_model_config.filename.iloc[0]
    print(file_test)
    th,score_test,model = get_metrics_prod(file_test,model,y_test.values.astype(float))
    print("threshold test:",th)
    print("score test:",score_test)
    return(model,param_config,th,score_test,model)  

In [240]:
results = []

In [7]:
outputs_metrics.sort_values('metric',ascending = False)#.query('model == "MLPClassifier"')

,Unnamed: 0,threshold,metric,filename,model,param_config,time
2447,2794,0.367672,1.0,"(7, 6, 49)_SVC_date_params.csv",SVC_date_params,49,7
2137,2763,0.367641,1.0,"(7, 6, 20)_SVC_date_params.csv",SVC_date_params,20,7
1365,2036,0.444142,1.0,"(5, 5, 41)_MLPClassifier_date_params.csv",MLPClassifier_date_params,41,5
1322,982,0.271605,1.0,"(2, 5, 38)_MLPClassifier_date_params.csv",MLPClassifier_date_params,38,2
1321,632,0.265734,1.0,"(1, 5, 38)_MLPClassifier_date_params.csv",MLPClassifier_date_params,38,1
...,...,...,...,...,...,...,...
2125,2062,0.408383,0.0,"(5, 6, 2)_SVC_date_params.csv",SVC_date_params,2,5
2124,1712,0.374134,0.0,"(4, 6, 2)_SVC_date_params.csv",SVC_date_params,2,4
2123,1362,0.338837,0.0,"(3, 6, 2)_SVC_date_params.csv",SVC_date_params,2,3
2122,1012,0.271571,0.0,"(2, 6, 2)_SVC_date_params.csv",SVC_date_params,2,2


In [243]:
i=0
    
for metric,(model,param_config) in best_config_p_metric.iteritems():
    if i>=1:
        print(metric)
        print(model)
        print(config)
        results.append( get_test_threshold(model,config))
    i+=1

average_std_25
MLPClassifier
34
Empty DataFrame
Columns: [Unnamed: 0, threshold, metric, filename, model, param_config, time]
Index: []


IndexError: single positional indexer is out-of-bounds

In [195]:
i=0
for model,row in best_config_p_metric.iterrows():
    print(model)
    if i>=7:
        for metric_name in row.index:
            print(metric_name)
            results.append( get_test_threshold(model,metric_name))
    i+=1
    

AttributeError: 'Series' object has no attribute 'iterrows'

In [23]:
import pickle 
filename = 'results_best_models.pkl'
filehandler = open(filename, 'wb') 
pickle.dump(results, filehandler)

,project_features_entity_id_all_grade_level_Grades35_sum,project_features_entity_id_all_grade_level_Grades68_sum,project_features_entity_id_all_grade_level_Grades912_sum,project_features_entity_id_all_grade_level_GradesPreK2_sum,project_features_entity_id_all_grade_level__NULL_sum,project_features_entity_id_all_poverty_level__NULL_sum,project_features_entity_id_all_poverty_level_highpov_sum,project_features_entity_id_all_poverty_level_highest_sum,project_features_entity_id_all_poverty_level_lowpove_sum,project_features_entity_id_all_poverty_level_moderate_sum,...,donation_features_entity_id_all_teacher_funding_rate_2yr_sum,donation_features_entity_id_all_teacher_funding_rate_2yr_imp,donation_features_entity_id_all_zip_avg_donations_1yr_sum,donation_features_entity_id_all_zip_avg_donations_1yr_imp,donation_features_entity_id_all_zip_avg_donations_2yr_sum,donation_features_entity_id_all_zip_avg_donations_2yr_imp,donation_features_entity_id_all_zip_funding_rate_1yr_sum,donation_features_entity_id_all_zip_funding_rate_1yr_imp,donation_features_entity_id_all_zip_funding_rate_2yr_sum,donation_features_entity_id_all_zip_funding_rate_2yr_imp
98741,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,355.57000,0.0,251.75000,0.0,0.500000,0.0,0.500000,0.0
98742,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.818182,0.0,316.27290,0.0,318.64944,0.0,0.941860,0.0,0.899543,0.0
98743,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,548.72000,0.0,419.73572,0.0,0.500000,0.0,0.625000,0.0
98744,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.00000,1.0,141.05000,0.0,0.000000,0.0,0.400000,0.0
98745,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.00000,1.0,357.49500,0.0,0.000000,0.0,0.357143,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138543,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,89.25000,0.0,106.25000,0.0,0.000000,0.0,0.000000,0.0
138544,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,21.25000,0.0,717.56665,0.0,0.000000,0.0,0.400000,0.0
138545,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.666667,0.0,219.34000,0.0,229.29000,0.0,0.727273,0.0,0.666667,0.0
138546,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500000,0.0,217.74342,0.0,238.70294,0.0,0.652174,0.0,0.523077,0.0
